In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from adjustText import adjust_text

import os,sys
import anndata as ad
import pandas as pd
import mudata as md
import glob
from tqdm import tqdm
from itertools import combinations
import itertools

from copy import deepcopy

import time

# Add the parent directory to the path to import from sibling directory
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../'))
from MultiStatePerturbSeqDataset import *
from merge_DE_results import parse_DE_results_2_adata

In [2]:
def get_reliability(x, sigma):
    sigma2_obs = np.var(x, ddof=1)  # Observed variance across features
    sigma2_sem = np.mean(sigma**2)   # Average squared standard error
    sigma2_true = sigma2_obs - sigma2_sem  # Estimated true signal variance
    reliability = sigma2_true / (sigma2_true + sigma2_sem)
    return(reliability)

def get_max_correlation(x_a, sigma_a, x_b, sigma_b):
    """Calculate maximum possible correlation"""
    rel_a = get_reliability(x_a, sigma_a)
    rel_b = get_reliability(x_b, sigma_b)
    return np.sqrt(rel_a * rel_b)

def get_lfc_correlation(x_a, sigma_a, x_b, sigma_b):
    """Calculate maximum possible correlation"""
    corr = x_a.corr(x_b)
    corr_ceil = get_max_correlation(x_a, sigma_a, x_b, sigma_b)
    return corr, corr_ceil

def _run_DE_test(pbulk_adata, test_state = 'Rest'):
    pbulk_adata.obs['log10_n_cells'] = np.log10(pbulk_adata.obs['n_cells'])

    n_donors = pbulk_adata.obs['donor_id'].nunique()
    if n_donors > 1:
        design_formula = '~ log10_n_cells + donor_id + target'
    else:
        design_formula = '~ log10_n_cells + target'

    # pbulk_adata = pbulk_adata[:, de_test_genes].copy()
    min_counts_per_gene = 10
    
    ms_perturb_data = MultistatePerturbSeqDataset(
        pbulk_adata,
        sample_cols = ['cell_sample_id'],
        perturbation_type = 'CRISPRi',
        target_col = 'perturbed_gene_id',
        sgrna_col = 'guide_id',
        state_col = 'culture_condition',
        control_level = 'NTC'
        )

    results = ms_perturb_data.run_target_DE(
        design_formula = design_formula,
        test_state = [test_state],
        min_counts_per_gene = min_counts_per_gene,
        return_model = False,
        n_cpus=8
        )

    n_cells_target = ms_perturb_data.adata.obs.groupby('target')['n_cells'].sum().reset_index()
    results = pd.merge(results.rename({'contrast':'target'}, axis=1), n_cells_target)
    results['n_donors'] = n_donors
    results['donors'] = '_'.join(pbulk_adata.obs['donor_id'].unique().tolist())
    results['signif'] = results['adj_p_value'] < 0.1
    return(results)

In [4]:
experiment_name = 'CD4i_final'
datadir = '../../../../../3_expts/processed_data'

In [5]:
sgrna_library_metadata = pd.read_csv('../../../metadata/sgRNA_library_curated.csv', index_col=0)
gene_name_to_guide_id = dict(zip(sgrna_library_metadata['sgrna_id'], sgrna_library_metadata['perturbed_gene_name']))
var_df = sc.read_h5ad(f'{datadir}/{experiment_name}/{experiment_name}_merged.DE_pseudobulk_corrected.h5ad', backed=True).var.copy()
de_counts = pd.read_csv(f'{datadir}/{experiment_name}/DE_results_all_confounders/DE_summary_stats_per_target.csv', index_col=0)
pbulk_adata = anndata.read_h5ad(f'{datadir}/{experiment_name}/{experiment_name}_merged.DE_pseudobulk_corrected.h5ad', backed=True)

In [6]:
#adata_de = sc.read_h5ad(datadir + '/CD4i_final/DE_results_all_confounders/CD4i_final.merged_DE_results_corrected.h5ad')
de_by_guide = pd.read_csv('../results/DE_by_guide.correlation_results.csv', index_col=0)

### Run Stim8hr

In [ ]:
cond = 'Stim8hr'
selected_perturbed_genes = de_by_guide[de_by_guide['culture_condition']==cond].target.tolist()
keep = (pbulk_adata.obs['perturbed_gene_name'].isin(selected_perturbed_genes + ['NTC'])) & (pbulk_adata.obs['culture_condition'] == cond)
pbulk_adata_test = pbulk_adata[keep].to_memory()
all_donors = pbulk_adata_test.obs['donor_id'].unique().tolist()

In [ ]:
chunk_size = 50
all_targets = pbulk_adata_test.obs['perturbed_gene_id'].unique().tolist()
all_targets.remove('NTC')

# Randomize targets before splitting (without replacement)
np.random.seed(42)
np.random.shuffle(all_targets)

# Split all_targets into groups based on chunk_size
target_chunks = [all_targets[i:i+chunk_size] for i in range(0, len(all_targets), chunk_size)]

# Initialize a binary matrix with zeros
target_chunk_matrix = pd.DataFrame(0, 
                                index=all_targets, 
                                columns=[f'chunk_{i}' for i in range(len(target_chunks))])

# Fill the matrix with 1s for each target in its respective chunk
for chunk_idx, chunk in enumerate(target_chunks):
    target_chunk_matrix.loc[chunk, f'chunk_{chunk_idx}'] = 1

In [ ]:
# Generate all possible combinations of 2 donors
donor_combinations = []
for train_donors in combinations(all_donors, 2):
    donor_combinations.append(
        list(train_donors)
    )

In [ ]:
for chunk_ix in range(len(target_chunks)):
    test_targets = target_chunk_matrix.index[target_chunk_matrix[f'chunk_{chunk_ix}'] == 1].tolist()
    pbulk_adata_test_for_de = pbulk_adata_test[pbulk_adata_test.obs.perturbed_gene_id.isin(test_targets+['NTC'])].copy()
    donor_robustness_results = pd.DataFrame()
    all_results_df = pd.DataFrame()
    for train_ds in donor_combinations:
        # Run DE tests
        results_df = _run_DE_test(pbulk_adata_test_for_de[pbulk_adata_test_for_de.obs['donor_id'].isin(train_ds)], test_state=cond)
        all_results_df = pd.concat([all_results_df, results_df])
    all_results_df.to_csv(f"./tmp_results/donor_robustness_results.{cond}.chunk_{chunk_ix}.csv.gz", compression='gzip')

### Run Rest

In [7]:
cond = 'Rest'
selected_perturbed_genes = de_by_guide[de_by_guide['culture_condition']==cond].target.tolist()
keep = (pbulk_adata.obs['perturbed_gene_name'].isin(selected_perturbed_genes + ['NTC'])) & (pbulk_adata.obs['culture_condition'] == cond)
pbulk_adata_test = pbulk_adata[keep].to_memory()
all_donors = pbulk_adata_test.obs['donor_id'].unique().tolist()

In [8]:
chunk_size = 50
all_targets = pbulk_adata_test.obs['perturbed_gene_id'].unique().tolist()
all_targets.remove('NTC')

# Randomize targets before splitting (without replacement)
np.random.seed(42)
np.random.shuffle(all_targets)

# Split all_targets into groups based on chunk_size
target_chunks = [all_targets[i:i+chunk_size] for i in range(0, len(all_targets), chunk_size)]

# Initialize a binary matrix with zeros
target_chunk_matrix = pd.DataFrame(0, 
                                index=all_targets, 
                                columns=[f'chunk_{i}' for i in range(len(target_chunks))])

# Fill the matrix with 1s for each target in its respective chunk
for chunk_idx, chunk in enumerate(target_chunks):
    target_chunk_matrix.loc[chunk, f'chunk_{chunk_idx}'] = 1

In [9]:
# Generate all possible combinations of 2 donors
donor_combinations = []
for train_donors in combinations(all_donors, 2):
    donor_combinations.append(
        list(train_donors)
    )

In [ ]:
for chunk_ix in range(6,len(target_chunks)):
    test_targets = target_chunk_matrix.index[target_chunk_matrix[f'chunk_{chunk_ix}'] == 1].tolist()
    pbulk_adata_test_for_de = pbulk_adata_test[pbulk_adata_test.obs.perturbed_gene_id.isin(test_targets+['NTC'])].copy()
    donor_robustness_results = pd.DataFrame()
    all_results_df = pd.DataFrame()
    for train_ds in donor_combinations:
        # Run DE tests
        results_df = _run_DE_test(pbulk_adata_test_for_de[pbulk_adata_test_for_de.obs['donor_id'].isin(train_ds)], test_state=cond)
        all_results_df = pd.concat([all_results_df, results_df])
    all_results_df.to_csv(f"./tmp_results/donor_robustness_results.{cond}.chunk_{chunk_ix}.csv.gz", compression='gzip')

/var/folders/qq/j023xkr13nxfrw65dx2bqwd00000gp/T/ipykernel_2572/3465598221.py:21: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  pbulk_adata.obs['log10_n_cells'] = np.log10(pbulk_adata.obs['n_cells'])
/Users/rzhu/miniconda3/envs/deseq/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Fitting dispersions...
... done in 0.05 seconds.

Fitting MAP dispersions...
... done in 0.04 seconds.

Fitting LFCs...
... done in 0.04 seconds.

Running Wald tests...
... done in 1.01 seconds.

Running Wald tests...


Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  377.911652        0.030427   0.254019  0.119783  0.904655   
ENSG00000000003    0.003822        0.141139  31.503094  0.004480  0.996425   
ENSG00000000419   63.140319       -0.377100   0.160638 -2.347512  0.018899   
ENSG00000000457   15.682999        0.085872   0.213971  0.401328  0.688179   
ENSG00000000460   37.050415       -0.164083   0.207918 -0.789171  0.430012   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.234381        1.343271   0.930464  1.443656  0.148836   
ENSG00000291110    0.081626       -0.276191   1.953500 -0.141383  0.887567   
ENSG00000291122    4.216023

... done in 1.04 seconds.



Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  377.911652       -0.025534   0.251660 -0.101461  0.919185   
ENSG00000000003    0.003822       -0.216689  31.502325 -0.006879  0.994512   
ENSG00000000419   63.140319       -0.066268   0.130845 -0.506462  0.612532   
ENSG00000000457   15.682999       -0.394067   0.169208 -2.328887  0.019865   
ENSG00000000460   37.050415        0.095402   0.175995  0.542069  0.587771   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.234381       -1.455401   1.752704 -0.830375  0.406327   
ENSG00000291110    0.081626       -0.080728   1.659832 -0.048636  0.961209   
ENSG00000291122    4.216023

Running Wald tests...
... done in 1.09 seconds.



...

Running Wald tests...
... done in 1.09 seconds.



Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  464.607657       -0.114966   0.178988 -0.642312  0.520671   
ENSG00000000003    0.004695       -0.040256  33.330175 -0.001208  0.999036   
ENSG00000000419   83.725082        0.075285   0.104025  0.723724  0.469235   
ENSG00000000457   21.786553        0.215052   0.166403  1.292357  0.196233   
ENSG00000000460   47.631807        0.076408   0.175942  0.434280  0.664085   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.088291       -0.321633   2.118677 -0.151809  0.879338   
ENSG00000291110    0.119879        0.985800   1.355448  0.727288  0.467050   
ENSG00000291122    5.916924

Running Wald tests...
... done in 1.11 seconds.



Log2 fold change & Wald test p-value, contrast vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
                   baseMean  log2FoldChange      lfcSE      stat    pvalue  \
CUSTOM001_PuroR  464.607657       -0.099522   0.179363 -0.554862  0.578989   
ENSG00000000003    0.004695       -0.072021  33.328835 -0.002161  0.998276   
ENSG00000000419   83.725082       -0.018135   0.099881 -0.181570  0.855920   
ENSG00000000457   21.786553        0.192115   0.152097  1.263105  0.206551   
ENSG00000000460   47.631807        0.160880   0.170015  0.946268  0.344012   
...                     ...             ...        ...       ...       ...   
ENSG00000291096    0.088291        0.346396   1.840031  0.188256  0.850676   
ENSG00000291110    0.119879        0.924421   1.355354  0.682051  0.495207   
ENSG00000291122    5.916924

/var/folders/qq/j023xkr13nxfrw65dx2bqwd00000gp/T/ipykernel_2572/3465598221.py:21: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  pbulk_adata.obs['log10_n_cells'] = np.log10(pbulk_adata.obs['n_cells'])


### Run Stim48hr

In [ ]:
cond = 'Stim48hr'
selected_perturbed_genes = de_by_guide[de_by_guide['culture_condition']==cond].target.tolist()
keep = (pbulk_adata.obs['perturbed_gene_name'].isin(selected_perturbed_genes + ['NTC'])) & (pbulk_adata.obs['culture_condition'] == cond)
pbulk_adata_test = pbulk_adata[keep].to_memory()
all_donors = pbulk_adata_test.obs['donor_id'].unique().tolist()

chunk_size = 50
all_targets = pbulk_adata_test.obs['perturbed_gene_id'].unique().tolist()
all_targets.remove('NTC')

# Randomize targets before splitting (without replacement)
np.random.seed(42)
np.random.shuffle(all_targets)

# Split all_targets into groups based on chunk_size
target_chunks = [all_targets[i:i+chunk_size] for i in range(0, len(all_targets), chunk_size)]

# Initialize a binary matrix with zeros
target_chunk_matrix = pd.DataFrame(0, 
                                index=all_targets, 
                                columns=[f'chunk_{i}' for i in range(len(target_chunks))])

# Fill the matrix with 1s for each target in its respective chunk
for chunk_idx, chunk in enumerate(target_chunks):
    target_chunk_matrix.loc[chunk, f'chunk_{chunk_idx}'] = 1

# Generate all possible combinations of 2 donors
donor_combinations = []
for train_donors in combinations(all_donors, 2):
    donor_combinations.append(
        list(train_donors)
    )

In [ ]:
for chunk_ix in range(len(target_chunks)):
    test_targets = target_chunk_matrix.index[target_chunk_matrix[f'chunk_{chunk_ix}'] == 1].tolist()
    pbulk_adata_test_for_de = pbulk_adata_test[pbulk_adata_test.obs.perturbed_gene_id.isin(test_targets+['NTC'])].copy()
    donor_robustness_results = pd.DataFrame()
    all_results_df = pd.DataFrame()
    for train_ds in donor_combinations:
        # Run DE tests
        results_df = _run_DE_test(pbulk_adata_test_for_de[pbulk_adata_test_for_de.obs['donor_id'].isin(train_ds)], test_state=cond)
        all_results_df = pd.concat([all_results_df, results_df])
    all_results_df.to_csv(f"./tmp_results/donor_robustness_results.{cond}.chunk_{chunk_ix}.csv.gz", compression='gzip')

### Parse results

In [ ]:
de_results_adatas = []
de_results_files = glob.glob(f'./tmp_results/donor_robustness_results.*.csv.gz')
for file in tqdm(de_results_files, desc="Processing DE result files"):
    # try:
    df = pd.read_csv(file, compression='gzip', index_col=0)
    df = df.rename({'target': 'target_contrast'}, axis=1)
    df['target_contrast_gene_name'] = df['target_contrast'].map(lambda x: gene_name_to_guide_id.get(x, x))
    donor_pairs = all_results_df['donors'].unique()
    for donor_pair in donor_pairs:
        temp_adata = parse_DE_results_2_adata(df[df['donors']==donor_pair])
        temp_adata.obs['donor_pair'] = donor_pair
        temp_adata.obs['chunk'] = int(re.search(r'chunk_(\d+)', file).group(1))
        de_results_adatas.append(temp_adata)
    # except EOFError:
    #     continue
combined_de_adata = anndata.concat(de_results_adatas)
combined_de_adata.obs_names = [item1+'_'+item2 for item1, item2 in zip(combined_de_adata.obs_names, combined_de_adata.obs['donor_pair'])]
combined_de_adata.var['gene_ids'] = combined_de_adata.var_names
combined_de_adata.var['gene_name'] = var_df.loc[combined_de_adata.var_names, 'gene_name']
assert combined_de_adata.obs_names.is_unique

In [ ]:
combined_de_adata.write_h5ad(f'{datadir}/{experiment_name}/{experiment_name}.DE_donor_robustness.h5ad')

In [ ]:
conditions = ['Rest', 'Stim8hr', 'Stim48hr']
all_correlations = pd.DataFrame()
for cond in conditions:
    de_results_files = glob.glob(f'./tmp_results/donor_robustness_results.{cond}.*.csv.gz')
    for file in de_results_files:
        
        all_results_df = pd.read_csv(file, index_col=0)
        donor_pairs = all_results_df['donors'].unique()
        comparison_pairs = []
        for pair1, pair2 in itertools.combinations(donor_pairs, 2):
            donors1 = set(pair1.split('_'))
            donors2 = set(pair2.split('_'))
            if donors1.isdisjoint(donors2):
                comparison_pairs.append((pair1, pair2))
        
        pivoted = all_results_df.pivot(columns='donors', index=['target','variable'], values='log_fc')
        pivoted_se = all_results_df.pivot(columns='donors', index=['target','variable'], values='lfcSE')
        pivoted_pval = all_results_df.pivot(columns='donors', index=['target','variable'], values='adj_p_value')
            
        correlations = pd.DataFrame()
        for target in tqdm(pivoted.index.get_level_values('target').unique()):
            target_data = pivoted.loc[target]
            target_se = pivoted_se.loc[target]
            target_p = pivoted_pval.loc[target]
                
            # Calculate correlations and F1 scores for specified pairs
            pair_correlations = []
            for pair in comparison_pairs:
                if pair[0] in target_data.columns and pair[1] in target_data.columns:
                    # Get mask for genes with p < 0.1 in either test
                    sig_mask = (target_p[pair[0]] < 0.2) | (target_p[pair[1]] < 0.2)
                    # sig_mask = (np.abs(target_data[pair[0]]) > 0.1) | (np.abs(target_data[pair[1]]) > 0.1)
                    
                    # Filter data using mask
                    data1 = target_data[pair[0]][sig_mask]
                    data2 = target_data[pair[1]][sig_mask]
                    se1 = target_se[pair[0]][sig_mask]
                    se2 = target_se[pair[1]][sig_mask]
                            
                    corr, corr_ceil = get_lfc_correlation(data1, se1, data2, se2)
            
                    pair_correlations.append({
                        'target': target,
                        'donors': pair[0],
                        'variable': pair[1],
                        'correlation': corr,
                        'correlation_ceiling': corr_ceil,
                        'n_signif': data1.shape[0],
                        'culture_condition': cond
                    })
            correlations = pd.concat([correlations, pd.DataFrame(pair_correlations)])
        
        correlations = pd.merge(correlations.rename({'target':'target_contrast'}, axis=1), de_counts[de_counts['condition']==cond], on='target_contrast', how='left')
        all_correlations = pd.concat([all_correlations, correlations])

In [87]:
all_correlations.to_csv('../results/Stim8hr_debyguidegenes_donor_robustness_for_followup_regulator_correlation.csv')

In [ ]:
donor_correlation_summary = pd.DataFrame({'donor_correlation_mean': all_correlations.groupby('target_name')['correlation'].mean(),
                                          'donor_correlation_min': all_correlations.groupby('target_name')['correlation'].min()})
donor_correlation_summary = pd.merge(donor_correlation_summary, de_counts[de_counts.condition=='Stim8hr'], on='target_name', how='left')
donor_correlation_summary.to_csv('../results/Stim8hr_debyguidegenes_donor_robustness_summary.csv')

In [92]:
all_correlations.iloc[150:200]

,target_contrast,donors,variable,correlation,correlation_ceiling,n_signif,culture_condition,target_name,condition,n_cells_target,n_up_genes,n_down_genes,n_total_de_genes,ontarget_effect_size,ontarget_significant,baseMean,offtarget_flag,n_total_genes_category,ontarget_effect_category
0,ENSG00000000457,CE0006864_CE0008162,CE0010866_CE0008678,0.301750,0.423777,1579,Stim8hr,SCYL3,Stim8hr,304.0,408,133,541,-5.650498,True,23.131029,False,>10 DE genes,on-target KD
1,ENSG00000000457,CE0006864_CE0010866,CE0008162_CE0008678,0.426270,0.560647,1149,Stim8hr,SCYL3,Stim8hr,304.0,408,133,541,-5.650498,True,23.131029,False,>10 DE genes,on-target KD
2,ENSG00000000457,CE0006864_CE0008678,CE0008162_CE0010866,0.351051,0.516720,1216,Stim8hr,SCYL3,Stim8hr,304.0,408,133,541,-5.650498,True,23.131029,False,>10 DE genes,on-target KD
3,ENSG00000005339,CE0006864_CE0008162,CE0010866_CE0008678,0.566845,0.720829,3843,Stim8hr,CREBBP,Stim8hr,1150.0,1969,1000,2969,-23.397890,True,58.522312,False,>10 DE genes,on-target KD
4,ENSG00000005339,CE0006864_CE0010866,CE0008162_CE0008678,0.587029,0.659984,3846,Stim8hr,CREBBP,Stim8hr,1150.0,1969,1000,2969,-23.397890,True,58.522312,False,>10 DE genes,on-target KD
5,ENSG00000005339,CE0006864_CE0008678,CE0008162_CE0010866,0.595337,0.628257,4296,Stim8hr,CREBBP,Stim8hr,1150.0,1969,1000,2969,-23.397890,True,58.522312,False,>10 DE genes,on-target KD
6,ENSG00000009307,CE0006864_CE0008162,CE0010866_CE0008678,0.632065,0.752549,1388,Stim8hr,CSDE1,Stim8hr,519.0,588,465,1053,-29.370139,True,176.271108,False,>10 DE genes,on-target KD
7,ENSG00000009307,CE0006864_CE0010866,CE0008162_CE0008678,0.715562,0.840855,1010,Stim8hr,CSDE1,Stim8hr,519.0,588,465,1053,-29.370139,True,176.271108,False,>10 DE genes,on-target KD
8,ENSG00000009307,CE0006864_CE0008678,CE0008162_CE0010866,0.459721,0.800648,1879,Stim8hr,CSDE1,Stim8hr,519.0,588,465,1053,-29.370139,True,176.271108,False,>10 DE genes,on-target KD
9,ENSG00000070061,CE0006864_CE0008162,CE0010866_CE0008678,0.457200,0.803168,1229,Stim8hr,ELP1,Stim8hr,166.0,429,399,828,-6.250322,True,36.853875,False,>10 DE genes,on-target KD


In [71]:
correlations

,target_contrast,donors,variable,correlation,correlation_ceiling,n_signif,culture_condition,target_name,condition,n_cells_target,n_up_genes,n_down_genes,n_total_de_genes,ontarget_effect_size,ontarget_significant,baseMean,offtarget_flag,n_total_genes_category,ontarget_effect_category
0,ENSG00000005812,CE0006864_CE0008162,CE0010866_CE0008678,0.104801,0.524101,1253,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD
1,ENSG00000005812,CE0006864_CE0010866,CE0008162_CE0008678,0.249726,0.824837,426,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD
2,ENSG00000005812,CE0006864_CE0008678,CE0008162_CE0010866,0.209743,0.796334,564,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD


In [77]:
all_correlations

,target_contrast,donors,variable,correlation,correlation_ceiling,n_signif,culture_condition,target_name,condition,n_cells_target,n_up_genes,n_down_genes,n_total_de_genes,ontarget_effect_size,ontarget_significant,baseMean,offtarget_flag,n_total_genes_category,ontarget_effect_category
0,ENSG00000005812,CE0006864_CE0008162,CE0010866_CE0008678,0.104801,0.524101,1253,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD
1,ENSG00000005812,CE0006864_CE0010866,CE0008162_CE0008678,0.249726,0.824837,426,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD
2,ENSG00000005812,CE0006864_CE0008678,CE0008162_CE0010866,0.209743,0.796334,564,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD
3,ENSG00000074621,CE0006864_CE0008162,CE0010866_CE0008678,0.543570,0.775809,2563,Stim8hr,SLC24A1,Stim8hr,650.0,1243,684,1927,0.000000,False,NaN,False,>10 DE genes,no on-target KD
4,ENSG00000074621,CE0006864_CE0010866,CE0008162_CE0008678,0.446443,0.709450,2652,Stim8hr,SLC24A1,Stim8hr,650.0,1243,684,1927,0.000000,False,NaN,False,>10 DE genes,no on-target KD
5,ENSG00000074621,CE0006864_CE0008678,CE0008162_CE0010866,0.485840,0.694601,2584,Stim8hr,SLC24A1,Stim8hr,650.0,1243,684,1927,0.000000,False,NaN,False,>10 DE genes,no on-target KD


In [67]:
de_counts[de_counts['condition']==cond]

,target_contrast,target_name,condition,n_cells_target,n_up_genes,n_down_genes,n_total_de_genes,ontarget_effect_size,ontarget_significant,baseMean,offtarget_flag,n_total_genes_category,ontarget_effect_category
0,ENSG00000012963,UBR7,Stim8hr,491.0,0,2,2,-12.952742,True,43.169196,True,2-10 DE genes,on-target KD
1,ENSG00000017260,ATP2C1,Stim8hr,469.0,0,1,1,-16.307246,True,102.399025,False,1 DE gene,on-target KD
2,ENSG00000067606,PRKCZ,Stim8hr,427.0,1,1,2,-1.658755,False,0.965897,False,2-10 DE genes,no on-target KD
3,ENSG00000092929,UNC13D,Stim8hr,830.0,0,2,2,-19.259466,True,60.904483,False,2-10 DE genes,on-target KD
4,ENSG00000100504,PYGL,Stim8hr,414.0,1,0,1,0.000000,False,NaN,False,1 DE gene,no on-target KD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33881,ENSG00000198363,ASPH,Stim8hr,391.0,0,1,1,-8.671965,True,19.861006,False,1 DE gene,on-target KD
33882,ENSG00000214013,GANC,Stim8hr,408.0,2,4,6,-6.159497,True,9.837870,True,2-10 DE genes,on-target KD
33883,ENSG00000230657,PRB4,Stim8hr,332.0,21,2,23,0.000000,False,NaN,False,>10 DE genes,no on-target KD
33884,ENSG00000251192,ZNF674,Stim8hr,245.0,1,1,2,-1.554408,False,1.399729,False,2-10 DE genes,no on-target KD


In [68]:
pd.merge(correlations.rename({'target':'target_contrast'}, axis=1), de_counts[de_counts['condition']==cond], on='target_contrast', how='left')

,target_contrast,donors,variable,correlation,correlation_ceiling,n_signif,culture_condition,target_name,condition,n_cells_target,n_up_genes,n_down_genes,n_total_de_genes,ontarget_effect_size,ontarget_significant,baseMean,offtarget_flag,n_total_genes_category,ontarget_effect_category
0,ENSG00000005812,CE0006864_CE0008162,CE0010866_CE0008678,0.104801,0.524101,1253,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD
1,ENSG00000005812,CE0006864_CE0010866,CE0008162_CE0008678,0.249726,0.824837,426,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD
2,ENSG00000005812,CE0006864_CE0008678,CE0008162_CE0010866,0.209743,0.796334,564,Stim8hr,FBXL3,Stim8hr,533.0,301,123,424,-20.399218,True,96.532976,False,>10 DE genes,on-target KD


In [ ]:
de_results_adatas = []
df = all_results_df.rename({'target': 'target_contrast'}, axis=1)
df['target_contrast_gene_name'] = df['target_contrast'].map(lambda x: gene_name_to_guide_id.get(x, x))
de_results_adatas.append(parse_DE_results_2_adata(df))

In [ ]:
de_results_files = glob.glob('./tmp_results/DE_results_by_guide.*.csv.gz')

In [ ]:
de_results_adatas = []

for file in tqdm(de_results_files, desc="Processing DE result files"):
    # try:
    df = pd.read_csv(file, compression='gzip', index_col=0)
    df = df.rename({'contrast': 'target_contrast'}, axis=1)
    df['target_contrast_gene_name'] = df['target_contrast'].map(lambda x: gene_name_to_guide_id.get(x, x))
    de_results_adatas.append(parse_DE_results_2_adata(df))
    # except EOFError:
    #     continue

combined_de_adata = anndata.concat(de_results_adatas, label='chunk')
# combined_de_adata.obs_names = combined_de_adata.obs_names.str.split('-').str[0]
assert combined_de_adata.obs_names.is_unique

In [ ]:
# Add gene names
combined_de_adata.var = var_df.loc[combined_de_adata.var_names]
combined_de_adata.layers['zscore'] = combined_de_adata.layers['log_fc'] / combined_de_adata.layers['lfcSE']
combined_de_adata.obs['n_cells_target'] = 1

In [ ]:
de_counts = get_de_stats(combined_de_adata, effect_col='zscore', signif_col='adj_p_value', alpha=0.1)
de_counts.head()

In [ ]:
combined_de_adata.write_h5ad(datadir + f'/DE_results_{run_name}/{experiment_name}.merged_DE_results_by_guide.h5ad')